In [63]:
import pandas as pd
import numpy as np
from mnist import MNIST
from tqdm import tqdm
from scipy.sparse.linalg import eigsh

In [40]:
images, labels = MNIST("data").load_testing()

In [41]:
images = np.array(images)

In [42]:
images.shape

(10000, 784)

In [49]:
def cos_sim(a,b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [50]:
sim_mat = np.zeros([10000,10000])
for i in tqdm(range(10000)):
    for j in range(i):
        sim_mat[i, j] = cos_sim(images[i], images[j])

100%|██████████| 10000/10000 [06:47<00:00, 24.55it/s]


In [52]:
sim_mat = sim_mat + sim_mat.T

In [56]:
threshold = np.min(np.max(sim_mat, axis=1))

In [59]:
diag = np.sum((sim_mat > threshold), axis=1)
D = np.zeros([10000,10000])
for i in range(10000):
    D[i,i] = diag[i]

L = D- sim_mat

In [61]:
diag2 = np.sum(sim_mat, axis=1)
D2 = np.zeros([10000,10000])
for i in range(10000):
    D2[i,i] = diag2[i]

L2 = D2- sim_mat

In [72]:
eigenvalues, eigenvectors = eigsh(L, k=32, which="SM")

In [82]:
eigenvalues2, eigenvectors2 = eigsh(L2, k=32, which="SM")

In [83]:
threshold1 = np.median(eigenvectors)
threshold2 = np.median(eigenvectors2)

In [84]:
eigenvectors_bin = np.where(eigenvectors > threshold1, 1, 0)
eigenvectors_bin2 = np.where(eigenvectors2 > threshold2, 1, 0)

In [78]:
eigenvectors_bin

array([[1, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 1, 1, 0],
       ...,
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 0],
       [1, 1, 0, ..., 1, 1, 0]])

In [86]:
print(len(np.unique(eigenvectors_bin,axis = 0)))
print(len(np.unique(eigenvectors_bin2,axis = 0)))

7006
2690


In [104]:
def map(eigenvectors_bin, labels, k):
    labels_dic = {i:0 for i in range(10)}
    labels_size_dic = {i:0 for i in range(10)}
    for i, eigenvector in enumerate(tqdm(eigenvectors_bin)):
        tp = 0
        fp = 0
        for j, eigenvector2 in enumerate(eigenvectors_bin):
            if np.abs(eigenvector - eigenvector2).sum() <= k:
                if labels[i] == labels[j]:
                    tp += 1
                else:
                    fp += 1
        labels_dic[labels[i]] += tp/(tp+fp)
        labels_size_dic[labels[i]] += 1
    return {i:labels_dic[i]/labels_size_dic[i] for i in range(10)}

In [103]:
ap_labels_dic

{0: 0.8812543211777673,
 1: 0.916076836286196,
 2: 0.8122052568993154,
 3: 0.6769046469764847,
 4: 0.7259522359035844,
 5: 0.6896376728468715,
 6: 0.8519151125387469,
 7: 0.8274702913639663,
 8: 0.5708846162665995,
 9: 0.652680105919939}